In [2]:
import openpyxl as pyxl
import os
from os.path import join
import psycopg2 as dbconn
from datetime import datetime as dt

hostname = "postgres-1.cvh49u2v99nl.ap-southeast-1.rds.amazonaws.com"
database = "postgres"
username = "sdcmktops"
password = "SDCsdc1234"


# Establish a connection to Postgresql
conn = dbconn.connect(
    host = hostname,
    database = database,
    user = username,
    password = password
)
cur = conn.cursor()

def parseXlsx (fileName, dirPath="."):

    # =====
    # Date related info
    # =====

    year = fileName[16:20]
    quarter = fileName[6:7]
    quarter_expr = fileName[6:9]

    wb = pyxl.load_workbook(filename=join(dirPath, fileName))

    for ws in wb.worksheets:
        if "VCData" + quarter_expr + "Quarter" + year in ws.title:
            break
    
    # =====
    # Quarterly data
    # =====
        
    bvp = ws['B6'].value
    avp = ws['B8'].value
    pns = ws['B10'].value
    prp = ws['B12'].value
    tcq_wd = ws['B15'].value
    tcq_we = ws['B16'].value
    bvp_wd = ws['B19'].value
    bvp_we = ws['B20'].value
    tvq_wd = ws['B23'].value
    tvq_we = ws['B24'].value

    # Query insert
    queryInsert = f"""
        INSERT INTO public.vesting_contract_quarter ("Year", "Quarter", "BVP", "AVP", "PNS", "PRP", "TCQ_WD", "TCQ_WE", "BVP_WD", "BVP_WE", "TVQ_WD", "TVQ_WE")
                                             VALUES ({year}, {quarter}, {bvp}, {avp}, {pns}, {prp}, {tcq_wd}, {tcq_we}, {bvp_wd}, {bvp_we}, {tvq_wd}, {tvq_we});
    """
    queryUpdate = f"""
        UPDATE public.vesting_contract_quarter 
        SET "Year"={year}, "Quarter"={quarter}, "BVP"={bvp}, "AVP"={avp}, "PNS"={pns}, "PRP"={prp}, "TCQ_WD"={tcq_wd}, "TCQ_WE"={tcq_we}, "BVP_WD"={bvp_wd}, "BVP_WE"={bvp_we}, "TVQ_WD"={tvq_wd}, "TVQ_WE"={tvq_we} 
        WHERE "Year"={year} AND "Quarter"={quarter};
    """
    
    cur.execute(f"""
        SELECT * FROM public.vesting_contract_quarter
        WHERE "Year"={year} AND "Quarter"={quarter};
    """)
    count = cur.fetchone()

    if count :
        cur.execute(queryUpdate)
        # print(queryUpdate)
    else:
        cur.execute(queryInsert)
        # print(queryInsert)

    # =====
    # Periodically data
    # =====

    for c in range(12):
        col = 2 + c
        period = 1 + c

        for r in range(4):
            row_wd = 27 + r*4
            row_we = 28 + r*4
            tcq_wd = ws.cell(row_wd, col).value
            tcq_wd = ws.cell(row_we, col).value

            # Query insert
            queryInsert = f"""
                INSERT INTO public.vesting_contract_period ("Year", "Quarter", "Period", "TCQ_WD", "TCQ_WE")
                                                    VALUES ({year}, {quarter}, {period}, {tcq_wd}, {tcq_we});
            """
            queryUpdate = f"""
                UPDATE public.vesting_contract_period 
                SET "Year"={year}, "Quarter"={quarter}, "Period"={period}, "TCQ_WD"={tcq_wd}, "TCQ_WE"={tcq_we}
                WHERE "Year"={year} AND "Quarter"={quarter} AND "Period"={period};
            """
            cur.execute(f"""
                SELECT * FROM public.vesting_contract_period
                WHERE "Year"={year} AND "Quarter"={quarter} AND "Period"={period};
            """)
            count = cur.fetchone()

            if count:
                cur.execute(queryUpdate)
                # print(queryUpdate)
            else:
                cur.execute(queryInsert)
                # print(queryInsert)
            # print(query)
            period += 12
    
    # =====
    # Public holiday dates
    # =====
            
    # Inspect maximum 10 rows of PH dates
    for r in range(10):
        date = ws[f'A{( 44 + r )}'].value
        if date == None: break
        
        date = dt.strftime(date, '%Y-%m-%d')

        queryInsert = f"""
            INSERT INTO public.vesting_contract_ph(
                "Year", "Quarter", "Date")
                VALUES ({year}, {quarter}, '{date}');
            """

        cur.execute(f"""
            SELECT * FROM public.vesting_contract_ph
            WHERE "Year"={year} AND "Quarter"={quarter} AND "Date"='{date}';
        """)
        count = cur.fetchone()

        if count:
            pass
        else:
            cur.execute(queryInsert)

    wb.close()
    
for fn in [f for f in os.listdir(".") if f.startswith("VCData") and f.endswith(".xlsx")]:
    parseXlsx(fn)

# Commit and close the connection
conn.commit()
conn.close()